DNA Foundation model for practice after learning the LLM NanoGPT / GPT2 from Dr. Andrej Kathpathy and Mistral DNA. 

** References **

<li> https://github.com/shreydan/makemore-series  </li>
<li> https://github.com/karpathy/nanoGPT  </li>
<li> https://github.com/raphaelmourad/Mistral-DNA  </li>



In [1]:
# conda activate torch_gpu
import os
import pandas as pd
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import GPT2LMHeadModel
from datasets import load_dataset


/mnt/data/tools/miniconda3/envs/torch_gpu_dna/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/data/tools/miniconda3/envs/torch_gpu_dna/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## Tokenization
To pretrain a model, we will use a file containig 100,000 non-overlapping DNA sequences of 200 bases corresponding to around 1% of the human genome (hg38 assembly). Mistral DNA works on Causal Language modelling (CLM) just like GPT not like BERT that works on MLM (Masked language model). In CLM, the token is predicted from the previous tokens. While in MLM, the token is masked irrespective of the position and all the tokens.

Tokenization is performed based on the Byte Pair Encoding (BPE). 

In [2]:
# First load the dataset
# Got the dataset from https://github.com/raphaelmourad/LLM-for-genomics-training
# dataset_text = load_dataset("csv", data_files="/mnt/data/projects/.immune/Personal/DNA-Language-Model/DNA_FM/data/genome_sequences/hg38/sequences_hg38_200b_verysmall.csv.gz")
savedir = "/mnt/data/projects/.immune/Personal/DNA-Language-Model/Mistral_DNA/"
os.chdir(savedir)
import pandas as pd
DNA_text = pd.read_csv(os.path.join(savedir,"data/genome_sequences/hg38/sequences_hg38_200b_verysmall.csv.gz"))

In [3]:
DNA_text[0:5]

,text
0,TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC...
1,CCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCC...
2,TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAA...
3,GAGGAGAACGCAACTCCGCCGTTGCAAAGGCGCGCCGCGCCGGCGC...
4,CACATGCTAGCGCGTCGGGGTGGAGGCGTGGCGCAGGCGCAGAGAG...


### Performing Tokenization

In [4]:
[ord(x)for x in DNA_text['text'][0]][0:5] ## These are basically the ASCII character

[84, 65, 65, 67, 67]

In [5]:
DNA_joined = "".join(DNA_text['text'].tolist())
DNA_joined[0:5]

'TAACC'

In [6]:
tokens = DNA_joined.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print('---')
print(DNA_joined[0:10])
print("length:", len(DNA_joined))
print('---')
print(tokens[0:10])
print("length:", len(tokens))

---
TAACCCTAAC
length: 19999800
---
[84, 65, 65, 67, 67, 67, 84, 65, 65, 67]
length: 19999800


In [7]:
print(max(tokens))
set(tokens) # ATGC

84


{65, 67, 71, 84}

In [8]:
### Combining the two character into one
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Pythonic way to iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
# print(stats)
print(sorted(((v,k) for k,v in stats.items()), reverse=True))

[(1549923, (67, 65)), (1547605, (84, 71)), (1516258, (65, 71)), (1515085, (67, 84)), (1504172, (67, 67)), (1502579, (71, 71)), (1454625, (84, 84)), (1441134, (65, 65)), (1225262, (84, 67)), (1224895, (71, 65)), (1191634, (71, 67)), (1086211, (65, 84)), (1015414, (65, 67)), (1014635, (71, 84)), (843065, (84, 65)), (367302, (67, 71))]


In [9]:
top_pair = max(stats, key=stats.get)
top_pair

(67, 65)

In [10]:
def merge(ids, pair, idx):
    new_text = []
    i = 0
    while i < len(ids):
        if i < len(ids) -1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            new_text.append(idx)
            i+=2
        else:
            new_text.append(ids[i])
            i+=1
    return new_text

# print(merge([5,5,6,6,7,8,6,7,9],[6,7],99))

tokens2 = merge(tokens, top_pair, 256)
print(tokens2[0:10])
print("length:", len(tokens2))

[84, 65, 65, 67, 67, 67, 84, 65, 65, 67]
length: 18449877


In [11]:
vocab_size = 4096 # the desired final vocabulary size
num_merges = vocab_size - 4 # since it is only A,T,G,C
ids = list(tokens) # copy so we don't destroy the original list

merges = {}
for i in range(num_merges):
    print(i)
    stats = get_stats(ids)
    pair = max(stats, key = stats.get)
    idx = 256 + i
    ids = merge(ids, pair, idx)
    merges[pair] = idx


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
print(len(ids), len(set(ids)))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

3721774 4093
compression ratio: 5.37X


In [13]:
def encode(text):
    token = text.encode("utf-8")
    token = list(map(int, token))
    return token

In [14]:
text = "ATGGCCTTAACCCCCCTCTGCGAATTACCATTGGGAGTTTCACCC"
token_encoded = encode(text)
print(len(token_encoded), len(text))
print(token_encoded, "\n",text)

45 45
[65, 84, 71, 71, 67, 67, 84, 84, 65, 65, 67, 67, 67, 67, 67, 67, 84, 67, 84, 71, 67, 71, 65, 65, 84, 84, 65, 67, 67, 65, 84, 84, 71, 71, 71, 65, 71, 84, 84, 84, 67, 65, 67, 67, 67] 
 ATGGCCTTAACCCCCCTCTGCGAATTACCATTGGGAGTTTCACCC


In [15]:
## Now UTF-8 is not 256 it is now 276. 
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

token_encoded = encode(text)
print(len(token_encoded), len(text))
print(token_encoded, "\n",text)

12 45
[308, 309, 259, 1493, 743, 337, 481, 293, 497, 256, 260, 67] 
 ATGGCCTTAACCCCCCTCTGCGAATTACCATTGGGAGTTTCACCC


In [16]:
len(token_encoded)

12

In [ ]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
value_voc =list(vocab.values())
print(value_voc[740:744], value_voc[100:104])

[b'AACAAA', b'TGGGGT', b'CAGGGT', b'CTGCG'] [b'd', b'e', b'f', b'g']


In [19]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [20]:
for i in range(len(token_encoded)):
    print(vocab[token_encoded[i]])

b'ATGG'
b'CCTT'
b'AA'
b'CCCCCCT'
b'CTGCG'
b'AATT'
b'ACCAT'
b'TGGG'
b'AGTTT'
b'CA'
b'CC'
b'C'


In [21]:
def decode(token):
    token_join = b''.join(vocab[idx] for idx in token)
    # print(token_join)
    text = token_join.decode("utf-8", errors = "replace") ## output not valid so we replace
    return text

In [22]:
text = decode(token_encoded)
print(text)

ATGGCCTTAACCCCCCTCTGCGAATTACCATTGGGAGTTTCACCC


In [23]:
print(decode(encode("ATGGCCTTAACC")))
text2 = decode(encode(text))
print(text2 == text)

ATGGCCTTAACC
True


In [25]:
print(b''.join(vocab[idx] for idx in token_encoded))

b'ATGGCCTTAACCCCCCTCTGCGAATTACCATTGGGAGTTTCACCC'


In [41]:
from collections import Counter
import math

class DNABert2LikeTokenizer:
    def __init__(
        self,
        vocab_size=4096,
        min_len=1,
        max_len=6,
        special_tokens=("[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]")
    ):
        self.vocab_size = vocab_size
        self.min_len = min_len
        self.max_len = max_len
        self.special_tokens = list(special_tokens)
        self.token2id = {}
        self.id2token = {}

    # -------------------------
    # Training (Unigram-style)
    # -------------------------
    def train(self, sequences):
        counts = Counter()

        for seq in sequences:
            seq = seq.upper()
            for i in range(len(seq)):
                for l in range(self.min_len, self.max_len + 1):
                    if i + l <= len(seq):
                        counts[seq[i:i+l]] += 1

        # Keep most frequent motifs
        vocab = [
            tok for tok, _ in counts.most_common(
                self.vocab_size - len(self.special_tokens)
            )
        ]

        # Build vocab
        full_vocab = self.special_tokens + vocab
        self.token2id = {tok: i for i, tok in enumerate(full_vocab)}
        self.id2token = {i: tok for tok, i in self.token2id.items()}

    # -------------------------
    # Tokenization
    # -------------------------
    def tokenize(self, sequence):
        sequence = sequence.upper()
        tokens = []
        i = 0

        while i < len(sequence):
            match = None
            for l in range(self.max_len, self.min_len - 1, -1):
                if i + l <= len(sequence):
                    substr = sequence[i:i+l]
                    if substr in self.token2id:
                        match = substr
                        break

            if match is None:
                tokens.append("[UNK]")
                i += 1
            else:
                tokens.append(match)
                i += len(match)

        return tokens

    def encode(self, sequence):
        tokens = ["[CLS]"] + self.tokenize(sequence) + ["[SEP]"]
        return [self.token2id.get(t, self.token2id["[UNK]"]) for t in tokens]

    def decode(self, ids):
        return [self.id2token[i] for i in ids]


In [42]:
sequences = [
    "ACGTACGTACGT",
    "CGTACGTACGTA",
    "ATATATATAT"
]

tokenizer = DNABert2LikeTokenizer(max_len=6)
tokenizer.train(sequences)

encoded = tokenizer.encode("ACGTACGT")
print(encoded)
print(tokenizer.decode(encoded))


[2, 31, 12, 3]
['[CLS]', 'ACGTAC', 'GT', '[SEP]']


In [ ]:
dataset_text['train'].data['text']

[
  [
    "TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTA (... 102 chars omitted)",
    "CCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAAACCCTAAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCAACCCCA (... 102 chars omitted)",
    "TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAGCCGGCCCGCCCGCCCG (... 102 chars omitted)",
    "GAGGAGAACGCAACTCCGCCGTTGCAAAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGCGCGCCGCGC (... 102 chars omitted)",
    "CACATGCTAGCGCGTCGGGGTGGAGGCGTGGCGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGACACATGCTACCGCGTCCAGGGGTG (... 102 chars omitted)",
    ...
    "CTGAACATAAATAATTCAAGTTTATATACTTTTGGCTTCCTGTTTTTACAGAGAGACTAAAGATATTTTGGCCCGTTAATAAACATGTTTTTTTCTGC (... 102 chars omitted)",
    "GTTTATAACCGTCTACTTCCTAGTTTTCTCTGGAAAATAAAAGATTACTAAGTATTAAAATTATAATCAATATATGTAAATAAAACTACTAGAAATAA (... 102 chars omitted)",
    "ATACAAATAAAAAGAGATACCTAACCTTCCCTGTGTTATATTTGTATGGGTAAAATGTTATGTTTTCAGAA

AttributeError: 'dict' object has no attribute 'text'

# Mistral DNA

In [61]:
import os
# import accelerate
# import flash_attn
import torch
import transformers
from datasets import load_dataset
from transformers import (
    AutoConfig, # load the configuration of pre-trained model. architecture and hyperparameter of the model
    AutoModelForCausalLM, # loads the pretrained causal language model for task like text generation
    AutoTokenizer, # load the tokenizer with a pre-trained model. convert the text to tokens
    DataCollatorForLanguageModeling, # designed for language modelling task. prepares batches for training by handling padding and masking
    EarlyStoppingCallback,  # is used to stop the training, if in the validation performance stops improving to save time and resources
    Trainer, # A high level API for training and evaluating the transformers. 
    TrainingArguments, # define the hyperparameter like learning rate, batch size, epoch, weight decay. 
)

# Choose the LLM architecture

Let’s look at the original archicture of Mixtral-8x7B-v0.1 which is stored in the data/models/Mixtral-8x7B-v0.1 folder Github https://github.com/raphaelmourad/Mistral-DNA/tree/main/data/models/Mixtral-8x7B-v0.1 

In [62]:
savedir = "/mnt/data/projects/.immune/Personal/DNA-Language-Model/Mistral_DNA/"
os.chdir(savedir)
config = AutoConfig.from_pretrained("data/models/Mixtral-8x7B-v0.1")

In [63]:
config

MixtralConfig {
  "_name_or_path": "data/models/Mixtral-8x7B-v0.1",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 256,
  "max_position_embeddings": 512,
  "model_type": "mixtral",
  "num_attention_heads": 8,
  "num_experts_per_tok": 1,
  "num_hidden_layers": 8,
  "num_key_value_heads": 8,
  "num_local_experts": 64,
  "output_router_logits": false,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "router_aux_loss_coef": 0.02,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 4096
}

By loading the configuration, we can inspect or modify the model’s architecture without loading the actual model weights. Let’s now initialize a causal language model from the loaded configuration object, with a specific attention implementation:


In [64]:
model = AutoModelForCausalLM.from_config(config, attn_implementation="eager")
# eager specifies the attention implementatin to use. Attention mechanism will be executed 
# eagerly which can be useful for debugging or when working with dynamic computation graphs
# Eager execution runs operations immediatedy as they are called in Python rather than adding 
# them to graph execution

In [65]:
model

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(4096, 256)
    (layers): ModuleList(
      (0-7): 8 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=256, out_features=256, bias=False)
          (k_proj): Linear(in_features=256, out_features=256, bias=False)
          (v_proj): Linear(in_features=256, out_features=256, bias=False)
          (o_proj): Linear(in_features=256, out_features=256, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=256, out_features=64, bias=False)
          (experts): ModuleList(
            (0-63): 64 x MixtralBlockSparseTop2MLP(
              (w1): Linear(in_features=256, out_features=256, bias=False)
              (w2): Linear(in_features=256, out_features=256, bias=False)
              (w3): Linear(in_features=256, out_features=256, bias=False)
              (act_fn): SiL

## Tokenization

In [66]:
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

/mnt/data/tools/miniconda3/envs/torch_gpu_dna/lib/python3.10/site-packages/huggingface_hub/utils/_validators.py:186: UserWarning: The `resume_download` argument is deprecated and ignored in `hf_hub_download`. Downloads always resume whenever possible.
  warnings.warn(


In [67]:
tokenizer

PreTrainedTokenizerFast(name_or_path='zhihan1996/DNABERT-2-117M', vocab_size=4096, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [89]:
tokens = tokenizer(
    "ATGGCCTTAACCCCCCTCTGCGAATTACCATTGGGAGTTTCACCC",
    return_tensors="pt"
)

print(tokens)
print(tokenizer.convert_ids_to_tokens(tokens["input_ids"][0]))


{'input_ids': tensor([[   1, 3218, 2819,  102,  435,  166,   24,  173,  117,  366,   13,    6,
            2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['[CLS]', 'ATG', 'GCCTTAA', 'CCCC', 'CCTCTG', 'CGAA', 'TTA', 'CCATT', 'GGGA', 'GTTTCA', 'CC', 'C', '[SEP]']


In [15]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
# hidden_states = model(inputs)[0] # [1, sequence_length, 768]

In [20]:
tokenizer.convert_ids_to_tokens(inputs[0])

['[CLS]',
 'A',
 'CGTA',
 'GCA',
 'TCGGA',
 'TCTATCTA',
 'TCGACA',
 'CTTGG',
 'TTA',
 'TCGA',
 'TCTA',
 'CGA',
 'GCA',
 'TCTC',
 'GTTA',
 'GC',
 '[SEP]']



PreTrainedTokenizerFast is a fast and efficient tokenizer used to process text data for the DNABERT-2-117M model. 

Here’s a breakdown of its configuration:<li>name_or_path='zhihan1996/DNABERT-2-117M': Specifies the name or path of the pre-trained tokenizer, indicating that it is associated with the DNABERT-2-117M model, which is designed for processing DNA sequences.</li>
<li>vocab_size=4096 Defines the size of the tokenizer’s vocabulary. 4**6 (ATGC) * 6 mer = 4096</li>

**Special_tokens:** Defines a set of special tokens used by the tokenizer: 
<li> unk_token: '[UNK]' - Represents unknown or out-of-vocabulary tokens.</li>
<li> sep_token: '[SEP]' - Used to separate segments within a sequence. </li>
<li> pad_token: '[PAD]' - Used for padding sequences to a uniform length. </li>
<li> cls_token: '[CLS]' - Typically used as the first token in a sequence to represent the classification token.</li>
<li> mask_token: '[MASK]' - Used in masked language modeling to hide tokens that the model must predict.</li>

</ul>
</li>
</ul>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-11"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<p>What do the other configuration parameters mean?</p>
<ol>
<li><code style="color: inherit">model_max_length=1000000000000000019884624838656</code></li>
<li><code style="color: inherit">is_fast=True</code></li>
<li><code style="color: inherit">padding_side='right'</code></li>
<li><code style="color: inherit">truncation_side='right'</code></li>
<li><code style="color: inherit">clean_up_tokenization_spaces=False</code></li>
<li><code style="color: inherit">added_tokens_decoder</code></li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-11"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-11" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>
<p><code style="color: inherit">model_max_length=1000000000000000019884624838656</code>: Represents the maximum length of sequences that the model can handle.</p>
<p>This extremely large value suggests that the model is designed to process very long sequences, although in practice, the actual limit will be constrained by available computational resources.</p>
</li>
<li><code style="color: inherit">is_fast=True</code>: Indicates that this tokenizer is optimized for speed, leveraging Rust-based implementations to accelerate tokenization processes.</li>
<li><code style="color: inherit">padding_side='right'</code>: Configures the tokenizer to pad sequences on the right side, ensuring that all sequences in a batch have the same length by adding padding tokens to the end of shorter sequences.</li>
<li><code style="color: inherit">truncation_side='right'</code>: Specifies that sequences will be truncated from the right side if they exceed the maximum length, preserving the beginning of the sequence.</li>
<li><code style="color: inherit">clean_up_tokenization_spaces=False</code>: Indicates that the tokenizer will not remove spaces after tokenization, preserving the original spacing in the text.</li>
<li><code style="color: inherit">added_tokens_decoder</code>: Maps token IDs to their corresponding <code style="color: inherit">AddedToken</code> objects, which include metadata such as whether the token is a special token and how it should be processed (e.g., stripping whitespace).</li>
</ol>
</blockquote>
</blockquote>
<p>This configuration ensures that the tokenizer is tailored to efficiently process DNA sequences, handling both the tokenization and padding/truncation of sequences in a manner that aligns with the model’s requirements.</p>
<p>By default, tokenizers may pad sequences on the right side (<code class="language-plaintext highlighter-rouge">padding_side='right'</code>). Let’s set the padding direction for the tokenizer.</p>


In [ ]:
# Tokenize Data based on BPE letter
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="longest", truncation=True, return_tensors="pt")

SyntaxError: unterminated string literal (detected at line 1) (1798563223.py, line 1)